In [2]:
import os

In [3]:
%pwd

'd:\\software\\python_vs\\Bone_marrow_survival_prediction\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\software\\python_vs\\Bone_marrow_survival_prediction'

### Entity

In [8]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path
    train_data_path: Path
    y_train_path :Path
    test_data_path : Path
    y_test_path : Path
    c : float
    kernel : str
    model_name : str
    target_column : str

### Configuration manager

In [24]:
from Bone_marrow_survival_prediction.constants import *
from Bone_marrow_survival_prediction.utils.comman import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath  = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH 
    ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.SupportVectorclassifier
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            y_train_path = config.y_train_path,
            y_test_path = config.y_test_path,
            c = params.c,
            kernel = params.kernel,
            model_name = config.model_name,
            target_column = schema
        )
        
        return model_trainer_config


### components

In [30]:
import pandas as pd
import os
from Bone_marrow_survival_prediction import logger
from sklearn.svm import SVC
import joblib


class ModelTrainer :
    def __init__(self, config:ModelTrainerConfig):
        self.config = config
        
    def train(self):
        x_train = pd.read_csv(self.config.train_data_path)
        y_train = pd.read_csv(self.config.y_train_path)
        x_test = pd.read_csv(self.config.test_data_path)
        y_test = pd.read_csv(self.config.y_test_path)
        
        
        svc = SVC(C=self.config.c, kernel = self.config.kernel,random_state=42)
        svc.fit(x_train,y_train)
        
        joblib.dump(svc,os.path.join(self.config.root_dir,self.config.model_name))
        
    
    


### Pipeline

In [31]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[ 2025-01-24 20:33:18,305: INFO: comman: yaml file: config\config.yaml loaded successfully]
[ 2025-01-24 20:33:18,307: INFO: comman: yaml file: params.yaml loaded successfully]
[ 2025-01-24 20:33:18,312: INFO: comman: yaml file: schema.yaml loaded successfully]
[ 2025-01-24 20:33:18,315: INFO: comman: created directory at: artifacts]
[ 2025-01-24 20:33:18,317: INFO: comman: created directory at: artifacts/model_trainer]


d:\software\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
